In [1]:
import sys 
sys.path.append('../utils')

from datetime import datetime 

from distilbert_utils import *

from distilbert_finetuning import * 

warnings.filterwarnings('ignore')

2022-05-03 20:22:10.331201: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64
2022-05-03 20:22:10.331248: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
GPU  = get_gpu()

Using GPU


In [4]:
labeled_examples, _ = get_sst_examples('./../../data/SST-2/train.tsv',test=False,discard_values = 0.92)
_, test_examples = get_sst_examples('./../../data/SST-2/dev.tsv', test=True,discard_values=1)

In [5]:
len(labeled_examples), len(test_examples)

(5428, 872)

In [6]:
label_map = {'0': 0, '1': 1}
train_examples = labeled_examples

In [7]:
from transformers import AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [8]:
transformer_type = 'distilbert-base-cased'

tokenizer = DistilBertTokenizer.from_pretrained(transformer_type)

train_dataloader = generate_data_loader(train_examples, label_map,tokenizer,batch_size =64, do_shuffle = True)

test_dataloader = generate_data_loader(test_examples, label_map,tokenizer)

In [9]:
#Create the tokenizer, model, optimizer, and criterion
model = transfer_device(GPU, DistilBertForSequenceClassification.from_pretrained(transformer_type))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier

In [10]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = binary_cross_entropy

In [11]:
#Train and save the model
model = train_model(GPU, train_dataloader, test_dataloader, tokenizer, model, optimizer, criterion,epochs =1 )

Pre-training validation loss: 0.7004958774362292 --- Accuracy: 0.4908256880733945

Epoch: 1on batch index: 13/14 ( 92.85714285714286% )
Training loss: 0.48098003794165217 --- Accuracy: 0.7431834929992631
Validation loss: 0.3342379001634462 --- Accuracy: 0.8589449541284404



In [14]:

print(f"distill_bert_finetuned_sst2_{len(train_examples)}_samples_{datetime.now():%Y-%m-%d_%H:%M:%S%z}.pt"
     )

distill_bert_finetuned_sst2_5428_samples_2022-05-03_20:59:04.pt


In [15]:
torch.save({
    'tokenizer': tokenizer,
    'distilbert': model.state_dict(),
}, f"distill_bert_finetuned_sst2_{len(train_examples)}_samples_{datetime.now():%Y-%m-%d_%H-%M-%S%z}.pt")